In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import seaborn as sns

import itertools


#For random forest for time series:
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

#For metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

#Having to import from subfolders of parent directories is a nightmare
import sys
import os
current = os.path.dirname(os.path.realpath("Single-House-Optimization.py"))
parent = os.path.dirname(current)
sys.path.append(parent+"\Functions")

#From Functions folder
from LoadSeries import loadSeries, moving_average
from Merge import merge
from Battery import Battery
from DPModel import DPModel
from Logic import logic_bat, logic_DP
from Funcs_Logic_DP import logic_rollout, get_price, print_price_summary, logic_series_print, policy_rollout, DP_stochastic

In [2]:
house = "h16"
prod = loadSeries("p",house,"prod_"+house,[24,48,168])[:-24]
cons = loadSeries("c",house,"cons_"+house,[24,48,168])[:-24]
cons

,cons_h16,cons_h16-24,cons_h16-48,cons_h16-168,Year,Month,Day,Weekday,Hour
2020-12-29 00:00:00,5.7,5.7,6.9,6.3,2020,12,29,1,0
2020-12-29 01:00:00,5.9,5.4,5.3,4.8,2020,12,29,1,1
2020-12-29 02:00:00,5.0,6.0,4.9,4.8,2020,12,29,1,2
2020-12-29 03:00:00,3.9,6.8,4.4,4.3,2020,12,29,1,3
2020-12-29 04:00:00,3.5,4.0,3.6,4.0,2020,12,29,1,4
...,...,...,...,...,...,...,...,...,...
2023-01-09 16:00:00,7.6,6.9,6.4,5.5,2023,1,9,0,16
2023-01-09 17:00:00,10.0,9.8,8.4,7.8,2023,1,9,0,17
2023-01-09 18:00:00,9.1,14.5,9.2,10.1,2023,1,9,0,18
2023-01-09 19:00:00,11.1,10.8,9.2,9.3,2023,1,9,0,19


In [13]:
N = 24*7
bat_capacity=13
Start="2021-06-19 00:00:00"
End = pd.date_range(start=Start, periods=N, freq="h")[-1]

BeforeProd = pd.date_range(end=Start, periods=2*7*24, freq="h")[0]
BeforeCons = pd.date_range(end=Start, periods=3000, freq="h")[0]

In [14]:
forecasterProd = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123,max_depth=8,max_features=3,n_estimators=100,n_jobs=4),
                 lags      = list(range(1,10+1,1))
                )
forecasterCons = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123,max_depth=8,max_features=4,n_estimators=100,n_jobs=4),
                 lags      = list(range(1,10+1,1))
                )


_, predProd = backtesting_forecaster(forecaster = forecasterProd,
                                     y          = prod.loc[BeforeProd:End].iloc[0:2*7*24+24]['prod_'+house],
                                     exog       = prod.loc[BeforeProd:End].iloc[0:2*7*24+24].drop(['prod_'+house], axis=1),
                                     initial_train_size = 2*7*24,
                                     steps      = 24,
                                     refit      = True,
                                     metric     = 'mean_squared_error',
                                     fixed_train_size   = True,
                                     verbose    = False
                                    )

_, predCons = backtesting_forecaster(forecaster = forecasterCons,
                                     y          = cons.loc[BeforeCons:End].iloc[0:3024]['cons_'+house],
                                     exog       = cons.loc[BeforeCons:End].iloc[0:3024].drop(['cons_'+house], axis=1),
                                     initial_train_size = 3000,
                                     steps      = 24,
                                     refit      = True,
                                     metric     = 'mean_squared_error',
                                     fixed_train_size   = True,
                                     verbose    = False
                                    )

In [22]:
merged = merge(house)
merged

,prod_h16,cons_h16,power_yield,SpotPriceDKK
2020-12-22 00:00:00,0.0,6.3,-6.3,11.090000
2020-12-22 01:00:00,0.0,4.8,-4.8,-0.150000
2020-12-22 02:00:00,0.0,4.8,-4.8,-55.880001
2020-12-22 03:00:00,0.0,4.3,-4.3,-72.919998
2020-12-22 04:00:00,0.0,4.0,-4.0,-53.720001
...,...,...,...,...
2023-01-09 16:00:00,0.1,7.6,-7.5,1204.510010
2023-01-09 17:00:00,0.0,10.0,-10.0,1296.579956
2023-01-09 18:00:00,0.0,9.1,-9.1,1354.890015
2023-01-09 19:00:00,0.0,11.1,-11.1,1266.089966


In [23]:
predProd=predProd.rename(columns={"pred":"prod_"+house})
predCons=predCons.rename(columns={"pred":"cons_"+house})
preds = pd.merge(predProd, predCons, how="outer", left_index=True,right_index=True)
preds["power_yield"] = preds["prod_"+house] - preds["cons_"+house]
preds["SpotPriceDKK"] = merged.loc[preds.index[0]:preds.index[-1]]["SpotPriceDKK"]
preds

,prod_h16,cons_h16,power_yield,SpotPriceDKK
2021-06-19 01:00:00,0.005895,5.021312,-5.015417,551.989990
2021-06-19 02:00:00,0.015841,4.242725,-4.226885,530.570007
2021-06-19 03:00:00,0.009385,3.965315,-3.955930,521.049988
2021-06-19 04:00:00,0.008046,3.841110,-3.833065,500.459991
2021-06-19 05:00:00,0.009743,3.801098,-3.791355,482.679993
2021-06-19 06:00:00,0.443260,4.083515,-3.640255,491.089996
2021-06-19 07:00:00,1.347322,4.293868,-2.946546,546.559998
2021-06-19 08:00:00,3.319039,5.058712,-1.739673,553.030029
2021-06-19 09:00:00,7.635055,5.932906,1.702149,482.829987
2021-06-19 10:00:00,10.465981,6.755063,3.710918,434.130005


In [24]:
battery = Battery(max_capacity=bat_capacity,max_charge=7)

series_battery = logic_rollout(merged.loc[Start:End].iloc[0:24].copy(), battery, logic_bat, get_price)

charge_list_bat = battery.charge_list

print_price_summary(series_battery)
logic_series_print(series_battery)

Cost for period: 2021-06-19 00:00:00 to 2021-06-19 23:00:00 is:  21.0  DKK
Average cost per year is: 7843.0 DKK
Number of kwh purchased in the period: 45.70000000001455
Average number of kwh purchased per year: 17417.660869570762
Average number of kwh sold per year: 40171.14782608918
hour     price    yield    surplus  buy      charge   before   after    cost     cumsum  
    0:   0.5962, -6.0000, -6.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.5770,  3.5770
    1:   0.5520, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  3.0359,  6.6129
    2:   0.5306, -4.6000, -4.6000,  0.0000,  0.0000,  0.0000,  0.0000,  2.4406,  9.0535
    3:   0.5210, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  2.1884, 11.2419
    4:   0.5005, -3.6000, -3.6000,  0.0000,  0.0000,  0.0000,  0.0000,  1.8017, 13.0436
    5:   0.4827, -3.9000, -3.9000,  0.0000,  0.0000,  0.0000,  0.0000,  1.8825, 14.9260
    6:   0.4911, -2.9000, -2.9000,  0.0000,  0.0000,  0.0000,  0.0000,  1.4242, 16.3502
    7:   

In [25]:
battery = Battery(max_capacity=bat_capacity,max_charge=7)

series_battery = logic_rollout(preds.loc[Start:End].iloc[0:24].copy(), battery, logic_bat, get_price)

charge_list_bat = battery.charge_list

print_price_summary(series_battery)
logic_series_print(series_battery)

Cost for period: 2021-06-19 01:00:00 to 2021-06-20 00:00:00 is:  21.0  DKK
Average cost per year is: 8138.0 DKK
Number of kwh purchased in the period: 46.4207062221147
Average number of kwh purchased per year: 17692.343945350327
Average number of kwh sold per year: 33982.585775876796
hour     price    yield    surplus  buy      charge   before   after    cost     cumsum  
    0:   0.5520, -5.0154, -5.0154,  0.0000,  0.0000,  0.0000,  0.0000,  2.7685,  2.7685
    1:   0.5306, -4.2269, -4.2269,  0.0000,  0.0000,  0.0000,  0.0000,  2.2427,  5.0111
    2:   0.5210, -3.9559, -3.9559,  0.0000,  0.0000,  0.0000,  0.0000,  2.0612,  7.0724
    3:   0.5005, -3.8331, -3.8331,  0.0000,  0.0000,  0.0000,  0.0000,  1.9183,  8.9907
    4:   0.4827, -3.7914, -3.7914,  0.0000,  0.0000,  0.0000,  0.0000,  1.8300, 10.8207
    5:   0.4911, -3.6403, -3.6403,  0.0000,  0.0000,  0.0000,  0.0000,  1.7877, 12.6084
    6:   0.5466, -2.9465, -2.9465,  0.0000,  0.0000,  0.0000,  0.0000,  1.6105, 14.2188
    7:   